# Spatial Correlation Analysis

#### Note: Use the epi-paint kernel

What is this used for?
- Calculates the spatial correlation for each localization with respect to another target. 
- Saves the list of the DoC values as a list. 

Workflow
- Define Folder of Cell to Analyse.
- Define the radius and step sizes that you are to calculate DoC for.
- Define protein species that are part of the analysis.
- Call each pair of protein species batchwise and calculate DoC for each localization. 
- Save the DoC values for each localization.

In [ ]:
# Import dependencies

import os as _os
import os.path as _ospath
import numpy as _np
import pandas as _pd
import h5py as _h5py
import yaml as _yaml
from PyQt5.QtWidgets import QMessageBox as _QMessageBox
import matplotlib.pyplot as _plt
from scipy.stats import spearmanr
from scipy.spatial import cKDTree
import itertools
from tqdm import tqdm

In [ ]:
# Define the folder location and the file extension inside the folder.

folder = '' # Folder name for specific cell.
min_radius = 100
step_size = 100
max_radius = 1000
folder = _ospath.join(folder, 'Masked')
file_extn = '.hdf5'
pixel_size = 130
file_names = [f for f in _os.listdir(folder) if f.endswith(file_extn)]

In [ ]:
# Define the output folder.

parent_folder, working_folder = _ospath.split(folder)
output_parent_folder = _ospath.join(parent_folder, 'Analysis', 'Correlations')
if not _ospath.exists(output_parent_folder):
    _os.makedirs(output_parent_folder)

In [ ]:
list_of_proteins = []

for file in file_names:
    protein = file.split('_')[0]
    if protein not in list_of_proteins:
        list_of_proteins.append(protein)

# Order the list of proteins
order = ['S2P', 'S5P', 'SC35', 'H3K4me3', 'H3K27ac', 'CTCF', 'H3K27me3', 'H3K9me3', 'Lamin']
list_of_proteins = sorted(list_of_proteins, key = order.index)

In [ ]:
# Define radii for analysis in nanometers
radii = _np.arange(min_radius, max_radius + step_size, step_size)
print(f'For every point, there will be {len(radii)} radii calculated!')

In [ ]:
def load_locs(path, qt_parent=None):
    with _h5py.File(path, "r") as locs_file:
        locs = locs_file["locs"][...]
    locs = _np.rec.array(
        locs, dtype=locs.dtype
    )  # Convert to rec array with fields as attributes
    info = load_info(path, qt_parent=qt_parent)
    return locs, info

class NoMetadataFileError(FileNotFoundError):
    pass

def load_info(path, qt_parent=None):
    path_base, path_extension = _ospath.splitext(path)
    filename = path_base + ".yaml"
    try:
        with open(filename, "r") as info_file:
            info = list(_yaml.load_all(info_file, Loader=_yaml.UnsafeLoader))
    except FileNotFoundError as e:
        print("\nAn error occured. Could not find metadata file:\n{}".format(filename))
        if qt_parent is not None:
            _QMessageBox.critical(
                qt_parent,
                "An error occured",
                "Could not find metadata file:\n{}".format(filename),
            )
        raise NoMetadataFileError(e)
    return info

def gradient_density(tree, spot, radii):
    distances, _ = tree.query(spot, k=len(tree.data), distance_upper_bound=max_radius)
    distances = distances[distances < max_radius]  # Filter valid distances
    densities = _np.array([_np.sum(distances <= r) / (_np.pi * r**2) for r in radii])
    return densities

def doc_calc(tree_a, tree_b, data_a, radii, protein_a, protein_b, output_parent_folder):
    doc_value = []
    for spot in tqdm(data_a, desc=f'Calculating DOC for {protein_a} vs {protein_b}'):
        density_1 = gradient_density(tree_a, spot, radii)
        density_2 = gradient_density(tree_b, spot, radii)
        corr = spearmanr(density_1, density_2) [0]
        if corr == _np.nan:
            corr = 0
        doc_value.append(corr)
    doc_value = _np.array(doc_value)
    output_folder = _ospath.join(output_parent_folder, str(min_radius) + '_' + str(step_size) + '_' + str(max_radius))
    if not _ospath.exists(output_folder):
        _os.makedirs(output_folder)
    _np.savetxt(_ospath.join(output_folder, f'{protein_a}_vs_{protein_b}.csv'), doc_value, delimiter=',', fmt='%.4f')
    return doc_value

def plot_doc_map(data, doc_values, protein_a, protein_b):
    _plt.figure(figsize=(8, 6))
    _plt.scatter(data[:, 0], data[:, 1], c=doc_values, cmap='coolwarm', marker='o', s=1)
    _plt.colorbar(label=f'DoC Score {protein_a} vs {protein_b}')
    _plt.title(f'DoC Map for {protein_a} vs {protein_b}')
    _plt.gca().invert_yaxis()
    _plt.show()

In [ ]:
correlation_matrix = _pd.DataFrame(index = list_of_proteins, columns = list_of_proteins, dtype = float)
counter = 0
for (i, file_1), (j, file_2) in itertools.combinations(enumerate(file_names), 2):
    protein_1 = file_1.split('_')[0]
    protein_2 = file_2.split('_')[0]
    # if protein_1 == 'SC35' or protein_2 == 'SC35': # Use this when the size of any protein sample is really large. # To Do: Parallelize the thread when large sample size is encountered.
    #     continue
    counter += 1
    print(f'Comparing {protein_1} and {protein_2}. ({counter}/{int(len(file_names)*(len(file_names)-1))/2:.0f})')
    locs_1, info_1 = load_locs(_ospath.join(folder, file_1))
    locs_2, info_2 = load_locs(_ospath.join(folder, file_2))
    data_1 = _np.column_stack((locs_1.x, locs_1.y))
    data_1 = data_1 * pixel_size
    data_2 = _np.column_stack((locs_2.x, locs_2.y))
    data_2 = data_2 * pixel_size
    tree_1 = cKDTree(data_1)
    tree_2 = cKDTree(data_2)
    # Calcualte DoC for Protein 1 vs Protein 2
    doc_1_vs_2 = doc_calc(tree_1, tree_2, data_1, radii, protein_1, protein_2, output_parent_folder)
    doc_2_vs_1 = doc_calc(tree_2, tree_1, data_2, radii, protein_2, protein_1, output_parent_folder)